# 1. Prototype With AWS Bedrock Foundational Models
Welcome to the first exercise in the CML Hands on Lab. In this notebook you will get familiar with calling an externally hosted foundational model. For this exercise we will be using AWS Bedrock service and the Anthropic Claude model hosted there.

![image](../assets/jupypter-session-bedrock.png)

### 1.1 Imports and AWS Bedrock client setup
Of note here is `boto3` SDK to interact with AWS services. The `get_bedrock_client` function is from AWS's [github repository](https://github.com/aws-samples/amazon-bedrock-workshop/blob/109ed616fd14c9eb26eda9bef96eb78c490d5ef6/utils/bedrock.py#L13). If you are running this code in your own environment, make sure to set AWS keys, preferably as environment variables.

In [20]:
import json
import os
from typing import Optional
import boto3
from botocore.config import Config

if os.environ.get("AWS_ACCESS_KEY_ID") == "":
    os.environ["AWS_ACCESS_KEY_ID"] = "<YOUR-ACCESS-KEY-ID>"   # Replace this if running in your own environment

if os.environ.get("AWS_SECRET_ACCESS_KEY") == "":
    os.environ["AWS_SECRET_ACCESS_KEY"] = "<YOUR-SECRET-ACCESS-KEY>"   # Replace this if running in your own environment

# TODO: for a lab, can reduce some of the checks in the below function
def get_bedrock_client(
    assumed_role: Optional[str] = None,
    endpoint_url: Optional[str] = None,
    region: Optional[str] = None,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    endpoint_url :
        Optional override for the Bedrock service API Endpoint. If setting this, it should usually
        include the protocol i.e. "https://..."
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if endpoint_url:
        client_kwargs["endpoint_url"] = endpoint_url

    bedrock_client = session.client(
        service_name="bedrock-runtime",
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

Then the client is initialized, binding to AWS region where Bedrock service is available. [As of October 2023](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-bedrock-asia-pacific-tokyo-aws-region/), these regions are us-east-1, us-west-2, and ap-northeast-1. We'll be using `us-east-1` as the default. This can be overwritten with an environment variable.

In [21]:
# Initializing the bedrock client using AWS credentials
# If you are using a special Assumed role or custom endpoint url, see get_bedrock_client
if os.environ.get("AWS_DEFAULT_REGION") == "":
    os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

boto3_bedrock = get_bedrock_client(
      region=os.environ.get("AWS_DEFAULT_REGION", None))

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


### 1.3 Set desired instruction: Text Summarization
The bedrock model shown in this notebook (Anthropic's Claude) is a general instruction-following text generation model. Meaning we can provide some instructions and input text to generate a response that will follow the instructions provided. As an example, we will provide instruction to the foundational model to summarize, in a few bullet points, a chunk of a text.  Model instructions typically follow a prescribed pattern and depend on the model used. In other words, the is no standard way to provide insturctions to different models. Below we follow [Anthropic's suggested structure](https://docs.anthropic.com/claude/docs/constructing-a-prompt). For example, note the use of keywords `Human:` and `Assistant:`. These are specific to the Claude foundational model. 

In [22]:
instruction_text = """Human: Please provide a summary of the text inside <text></text> XML tags. Do not add any information that is not mentioned in this text. 
                             Provide no more than 3 bullet points in the summary, each being a complete sentece. 
                             Start your summary with simply saying "Here's a brief summary of the provided text:". 
                    <text>{{USER_TEXT}}</text>
                    Assistant:"""

### 1.4 Set input text and create complete prompt <a id='1.4'></a>

This is the input text that we want to be summarized. The length of this text plus any included instructions must fit within the context window size of the selected model. For Claude it's approximately 9,000 words.

In [23]:
input_text = '''Machine learning has become one of the most critical capabilities for modern businesses to grow and stay competitive today. From automating internal processes to optimizing the design, creation, and marketing processes behind virtually every product consumed, ML models have permeated almost every aspect of our work and personal lives.
ML development is iterative and complex, made even harder because most ML tools aren’t built for the entire machine learning lifecycle. Cloudera Machine Learning on Cloudera Data Platform accelerates time-to-value by enabling data scientists to collaborate in a single unified platform that is all inclusive for powering any AI use case. Purpose-built for agile experimentation and production ML workflows, Cloudera Machine Learning manages everything from data preparation to MLOps, to predictive reporting. Solve mission critical ML challenges along the entire lifecycle with greater speed and agility to discover opportunities which can mean the difference for your business.
Each ML workspace enables teams of data scientists to develop, test, train, and ultimately deploy machine learning models for building predictive applications all on the data under management within the enterprise data cloud. ML workspaces support fully-containerized execution of Python, R, Scala, and Spark workloads through flexible and extensible engines.'''

# Replace instruction placeholder to build a complete prompt
full_prompt = instruction_text.replace("{{USER_TEXT}}", input_text)

### 1.5 Creating API request for Titan model
With the prompt out of the way, we generate a JSON payload to send to Bedrock for processing. The parameters and format required for this API request is specific to the Titan model, see AWS Bedrock documentation for more details.

In [24]:
# Model expects a JSON object with a defined schema
body = json.dumps({"prompt": full_prompt,
             "max_tokens_to_sample":4096,
             "temperature":0.6,
             "top_k":250,
             "top_p":1.0,
             "stop_sequences":[]
              })

# Provide a model ID and call the model with the JSON payload
modelId = 'anthropic.claude-v2:1'
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept='application/json', contentType='application/json')
response_body = json.loads(response.get('body').read())
print("Model results successfully retreived")

Model results successfully retreived


### 1.6 Review the results
The response body is specific to the Claude Model API, see AWS Bedrock documentation for more details. 

In [25]:
result = response_body.get('completion')
print(result)

 Here's a brief summary of the provided text:

- Machine learning models have become critical for modern businesses to stay competitive and grow.
- ML development is complex and iterative, made harder because most ML tools aren’t built for the entire machine learning lifecycle. 
- Cloudera Machine Learning aims to accelerate time-to-value by enabling collaboration on a unified platform for the entire ML lifecycle.


**(BONUS)** Go back to [step 1.4](#1.4) and paste a different text for the model to summarize. See how it does on the task. 

### 1.7 Takeaways

* [Cloudera Machine Learning](https://docs.cloudera.com/machine-learning/cloud/product/topics/ml-product-overview.html#cdsw_overview) provides a flexible environment to integrate with 3rd party foundational models
* JupyterLabs is a supported editor, along with other editors that can be optinoally added to [custom runtimes](https://docs.cloudera.com/machine-learning/cloud/runtimes/topics/ml-creating-a-customized-runtimes-image.html) (e.g. RStudio, VSCode)  
* Users can prototype LLM solutions quickly, using development tools they are most efficient with

### Up Next: Go to Exercise 2